In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:

E_B = 100
P_Battery_Max = 50
Battery_lifetime = 10*365 #忽略闰年
Cost_Battery = P_Battery_Max*800+E_B*1800
Cost_Battery_daily = Cost_Battery/(Battery_lifetime)

init_SOC=0.5  # assumption

SOC_UP =  soc_upper_bound = 0.9
SOC_FLOOR = soc_floor_bound = 0.1
eta_Bc=eta_Bd=0.95

# 读取数据
data_A = pd.read_csv("data/园区A.csv")
data_B = pd.read_csv("data/园区B.csv")
data_C = pd.read_csv("data/园区C.csv")

P_user_A = data_A["用电量"]
P_user_B = data_B["用电量"]
P_user_C = data_C["用电量"]

P_PV_A = data_A["光伏供电量"]
P_WG_B = data_B["风电供电量"]
P_PV_C = data_C["光伏供电量"]
P_WG_C = data_C["风电供电量"]


In [16]:
# C
def setSOC(SOC,t,value):
    SOC[t]=value
def getSOC(SOC,t):
    if t==-1:
        return SOC[-1]
    return SOC[t]
def gamma_B(P_B):
    if P_B<0:
        return eta_Bc
    else:
        return 1/eta_Bd
def get_P_B(Delta_P):#限制充放电功率
    P_Bdmax = P_Bcmax = P_Battery_Max
    if Delta_P>-P_Bdmax and Delta_P<=P_Bcmax:
        return Delta_P
    elif Delta_P>P_Bcmax:
        return P_Bcmax
    else:
        return -P_Bdmax
# 放电时为正，充电时为负
def cal_P_B(SOC,P_Battery_Max,Delta_P,t,debug=True):
    P_B=get_P_B(Delta_P) 
    SOC_last = getSOC(SOC,t) 
    gamma_B_value = gamma_B(P_B)
    SOC_new = SOC_last-gamma_B_value*P_B*1/E_B
    
    # 限制充放电容量
    if SOC_new > SOC_UP:
        SOC_new = SOC_UP
    elif SOC_new < SOC_FLOOR:
        SOC_new = SOC_FLOOR
    P_B_Battery = (SOC_last-SOC_new)*E_B
    P_B_use =(SOC_last-SOC_new)*E_B/gamma_B(P_B)/1
    if debug:
        if(P_B_use>0):
            print(f"外部放电{abs(P_B_use)}kW,消耗电池{abs(P_B_Battery)}")
        elif(P_B_use<0):
            print(f"外部充电{abs(P_B_use)}kW,补充电池{abs(P_B_Battery)}")
    return P_B_use,P_B_Battery
def find_lack(t):
    data = P_PV_C+P_WG_C-P_user_C
    for i in range(t+1,len(data)):
        if data[i] < 0:
            return i
    return -1
def wait_sun(P_wind,C_wind,Battery,SOC,t,debug):
    j = t
    n = find_lack(t)
    lack = P_user_C[n]-P_PV_C[n]-P_WG_C[n]
    extra=0
    for k in range(j+1,n):
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k],t,debug)
        extra+=abs(P_B_use)
    B_empty = (SOC_UP-getSOC(SOC,t))*E_B
    if(lack-extra>B_empty):
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t,debug)
        P_wind+=P_B_use
        Battery-=P_B_Battery 
        C_wind=-P_B_use 
        return P_wind,C_wind,Battery 
    while(n<=23):
        j = n+1
        n = find_lack(n)
        if n==-1:
            break
        lack += P_user_C[n]-P_PV_C[n]-P_WG_C[n]
        extra=0
        for k in range(j+1,n):
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k],t,debug)
            extra+=abs(P_B_use)
        B_empty = (SOC_UP-getSOC(SOC,t))*E_B
        if(lack-extra>B_empty):
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t,debug) 
            P_wind+=P_B_use
            Battery-=P_B_Battery 
            C_wind=-P_B_use 
            return P_wind,C_wind,Battery 
        # 更新电池
        # Battery+=lack-extra
    return P_wind,C_wind,Battery 

def debug_Battery(Battery,SOC,t,debug):
    if debug:    
        print(f"电池电量：{Battery},{SOC[t]}")


# Battery = init_SOC*E_B or 0
def Process(SOC,Battery,P_user, P_light, P_wind,label,t,debug=False):
    if debug:
        print(f"现在是{t}:00,电池电量：{Battery}")
    P_wind = P_wind
    C_light=0
    C_wind=0 
    C_net=0

    # 光
    if P_light > P_user:
        C_light = P_user
        P_user = 0
        P_light -= C_light
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_light,t,debug)
        # charge
        P_light+=P_B_use 
        Battery-=P_B_Battery
        debug_Battery(Battery,SOC,t,debug)
    else:
        C_light = P_light
        P_user -= P_light
    
    if P_user == 0:
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery
    
    # 电池
    # discharge
    if(debug):
        print("自发电不足，动用电池放电")
    P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,P_user,t,debug) #放电时为正，充电时为负
    Battery -= P_B_Battery 
    debug_Battery(Battery,SOC,t,debug)

    P_user -=P_B_use 
    if P_user==0:
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t,debug) 
        P_wind+=P_B_use
        Battery-=P_B_Battery 
        debug_Battery(Battery,SOC,t,debug)
        C_wind=-P_B_use 
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery
    
    # 风
    if P_wind > P_user:
        C_wind = P_user
        P_user = 0
        P_wind-=C_wind

        if label == 'B':
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t,debug)
            C_wind -= P_B_use
            P_wind +=P_B_use 
            Battery-=P_B_Battery  
            debug_Battery(Battery,SOC,t,debug)
            
        if label == 'C':
            # 额外考虑
            P_wind,C_wind,Battery = wait_sun(P_wind,C_wind,Battery,SOC,t,debug)
            debug_Battery(Battery,SOC,t,debug)
            
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery

        
    else:
        C_wind = P_wind
        P_user -= P_wind
    
    if P_user > 0:
        C_net = P_user
        P_user = 0
    # SOC[t]=Battery/E_B
    SOC.append(Battery/E_B)
    
    return C_light, C_wind, C_net,Battery

In [15]:
#调参 A

#初值
E_B = 100
P_Battery_Max = 50
E_B_change=[]
P_Battery_Max_change=[]
cost_All=[]
cost_PV=[]
cost_WG=[]
cost_net=[]

while E_B<=10000:
    P_Battery_Max=50
    while P_Battery_Max<=10000 and P_Battery_Max<=0.8*E_B:
        E_B_change.append(E_B)
        P_Battery_Max_change.append(P_Battery_Max)
        Battery = init_SOC*E_B
        cost= []
        Battery_lifetime = 10*365 #忽略闰年
        Cost_Battery = P_Battery_Max*800+E_B*1800
        Cost_Battery_daily = Cost_Battery/(Battery_lifetime)
        sum_All = Cost_Battery_daily
        sum_PV = 0
        sum_WG = 0
        sum_net = 0
        SOC=[init_SOC]
        print(E_B,P_Battery_Max)
        for t in range(0,24):
            C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_A[t], P_PV_A[t],0,"A",t,debug=False) 
            cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
            sum_PV+=C_light*0.4
            sum_WG+=C_wind*0.5
            sum_net+=C_net*1
            sum_All+=cost[t]

        cost_All.append(sum_All)
        cost_PV.append(sum_PV)
        cost_WG.append(sum_WG)
        cost_net.append(sum_net)
        df = pd.DataFrame({"E_B":E_B_change,"P_Battery_Max":P_Battery_Max_change,"cost_ALL":cost_All,"cost_PV":cost_PV,"cost_WG":cost_WG,"cost_net":cost_net})
        df.to_csv("data/parameter_change_A.csv",index=False)
        P_Battery_Max+=50
    E_B+=100
    #print(E_B)
### bug 上面这个循环死循环，C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_C[t], P_PV_C[t],P_WG_C[t],"C",t,debug=False) 会报错
df = pd.DataFrame({"E_B":E_B_change,"P_Battery_Max":P_Battery_Max_change,"cost_ALL":cost_All,"cost_PV":cost_PV,"cost_WG":cost_WG,"cost_net":cost_net})
df.to_csv("data/parameter_change_A.csv",index=False)

100 50
200 50
200 100
200 150
300 50
300 100
300 150
300 200
400 50
400 100
400 150
400 200
400 250
400 300
500 50
500 100
500 150
500 200
500 250
500 300
500 350
500 400
600 50
600 100
600 150
600 200
600 250
600 300
600 350
600 400
600 450
700 50
700 100
700 150
700 200
700 250
700 300
700 350
700 400
700 450
700 500
700 550
800 50
800 100
800 150
800 200
800 250
800 300
800 350
800 400
800 450
800 500
800 550
800 600
900 50
900 100
900 150
900 200
900 250
900 300
900 350
900 400
900 450
900 500
900 550
900 600
900 650
900 700
1000 50
1000 100
1000 150
1000 200
1000 250
1000 300
1000 350
1000 400
1000 450
1000 500
1000 550
1000 600
1000 650
1000 700
1000 750
1000 800
1100 50
1100 100
1100 150
1100 200
1100 250
1100 300
1100 350
1100 400
1100 450
1100 500
1100 550
1100 600
1100 650
1100 700
1100 750
1100 800
1100 850
1200 50
1200 100
1200 150
1200 200
1200 250
1200 300
1200 350
1200 400
1200 450
1200 500
1200 550
1200 600
1200 650
1200 700
1200 750
1200 800
1200 850
1200 900
1200 950


In [19]:
#调参 B

#初值
E_B = 100
P_Battery_Max = 50
E_B_change=[]
P_Battery_Max_change=[]
cost_All=[]
cost_PV=[]
cost_WG=[]
cost_net=[]

while E_B<=10000:
    P_Battery_Max=50
    while P_Battery_Max<=10000 and P_Battery_Max<=0.8*E_B:
        E_B_change.append(E_B)
        P_Battery_Max_change.append(P_Battery_Max)
        Battery = init_SOC*E_B
        cost= []
        Battery_lifetime = 10*365 #忽略闰年
        Cost_Battery = P_Battery_Max*800+E_B*1800
        Cost_Battery_daily = Cost_Battery/(Battery_lifetime)
        sum_All = Cost_Battery_daily
        sum_PV = 0
        sum_WG = 0
        sum_net = 0
        SOC=[init_SOC]
        print(E_B,P_Battery_Max)
        for t in range(0,24):
            C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_B[t], 0,P_WG_B[t],"B",t,debug=False) 
            cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
            
            sum_PV+=C_light*0.4
            sum_WG+=C_wind*0.5
            sum_net+=C_net*1
            sum_All+=cost[t]

        cost_All.append(sum_All)
        cost_PV.append(sum_PV)
        cost_WG.append(sum_WG)
        cost_net.append(sum_net)
        df = pd.DataFrame({"E_B":E_B_change,"P_Battery_Max":P_Battery_Max_change,"cost_ALL":cost_All,"cost_PV":cost_PV,"cost_WG":cost_WG,"cost_net":cost_net})
        df.to_csv("data/parameter_change_B.csv",index=False)
        P_Battery_Max+=50
    E_B+=100
    #print(E_B)
### bug 上面这个循环死循环，C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_C[t], P_PV_C[t],P_WG_C[t],"C",t,debug=False) 会报错
df = pd.DataFrame({"E_B":E_B_change,"P_Battery_Max":P_Battery_Max_change,"cost_ALL":cost_All,"cost_PV":cost_PV,"cost_WG":cost_WG,"cost_net":cost_net})
df.to_csv("data/parameter_change_B.csv",index=False)

100 50
200 50
200 100
200 150
300 50
300 100
300 150
300 200
400 50
400 100
400 150
400 200
400 250
400 300
500 50
500 100
500 150
500 200
500 250
500 300
500 350
500 400
600 50
600 100
600 150
600 200
600 250
600 300
600 350
600 400
600 450
700 50
700 100
700 150
700 200
700 250
700 300
700 350
700 400
700 450
700 500
700 550
800 50
800 100
800 150
800 200
800 250
800 300
800 350
800 400
800 450
800 500
800 550
800 600
900 50
900 100
900 150
900 200
900 250
900 300
900 350
900 400
900 450
900 500
900 550
900 600
900 650
900 700
1000 50
1000 100
1000 150
1000 200
1000 250
1000 300
1000 350
1000 400
1000 450
1000 500
1000 550
1000 600
1000 650
1000 700
1000 750
1000 800
1100 50
1100 100
1100 150
1100 200
1100 250
1100 300
1100 350
1100 400
1100 450
1100 500
1100 550
1100 600
1100 650
1100 700
1100 750
1100 800
1100 850
1200 50
1200 100
1200 150
1200 200
1200 250
1200 300
1200 350
1200 400
1200 450
1200 500
1200 550
1200 600
1200 650
1200 700
1200 750
1200 800
1200 850
1200 900
1200 950


In [17]:
#调参 C

#初值
E_B = 100
P_Battery_Max = 50
E_B_change=[]
P_Battery_Max_change=[]
cost_All=[]
cost_PV=[]
cost_WG=[]
cost_net=[]

while E_B<=10000:
    P_Battery_Max=50
    while P_Battery_Max<=10000 and P_Battery_Max<=0.8*E_B:
        E_B_change.append(E_B)
        P_Battery_Max_change.append(P_Battery_Max)
        Battery = init_SOC*E_B
        cost= []
        Battery_lifetime = 10*365 #忽略闰年
        Cost_Battery = P_Battery_Max*800+E_B*1800
        Cost_Battery_daily = Cost_Battery/(Battery_lifetime)
        sum_All = Cost_Battery_daily
        sum_PV = 0
        sum_WG = 0
        sum_net = 0
        SOC=[init_SOC]
        print(E_B,P_Battery_Max)
        for t in range(0,24):
            C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_C[t], P_PV_C[t],P_WG_C[t],"C",t,debug=False) 
            cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
            
            sum_PV+=C_light*0.4
            sum_WG+=C_wind*0.5
            sum_net+=C_net*1
            sum_All+=cost[t]

        cost_All.append(sum_All)
        cost_PV.append(sum_PV)
        cost_WG.append(sum_WG)
        cost_net.append(sum_net)
        df = pd.DataFrame({"E_B":E_B_change,"P_Battery_Max":P_Battery_Max_change,"cost_ALL":cost_All,"cost_PV":cost_PV,"cost_WG":cost_WG,"cost_net":cost_net})
        df.to_csv("data/parameter_change_C.csv",index=False)
        P_Battery_Max+=50
    E_B+=100
    #print(E_B)
### bug 上面这个循环死循环，C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_C[t], P_PV_C[t],P_WG_C[t],"C",t,debug=False) 会报错
df = pd.DataFrame({"E_B":E_B_change,"P_Battery_Max":P_Battery_Max_change,"cost_ALL":cost_All,"cost_PV":cost_PV,"cost_WG":cost_WG,"cost_net":cost_net})
df.to_csv("data/parameter_change_C.csv",index=False)

100 50
200 50
200 100
200 150
300 50
300 100
300 150
300 200
400 50
400 100
400 150
400 200
400 250
400 300
500 50
500 100
500 150
500 200
500 250
500 300
500 350
500 400
600 50
600 100
600 150
600 200
600 250
600 300
600 350
600 400
600 450
700 50
700 100
700 150
700 200
700 250
700 300
700 350
700 400
700 450
700 500
700 550
800 50
800 100
800 150
800 200
800 250
800 300
800 350
800 400
800 450
800 500
800 550
800 600
900 50
900 100
900 150
900 200
900 250
900 300
900 350
900 400
900 450
900 500
900 550
900 600
900 650
900 700
1000 50
1000 100
1000 150
1000 200
1000 250
1000 300
1000 350
1000 400
1000 450
1000 500
1000 550
1000 600
1000 650
1000 700
1000 750
1000 800
1100 50
1100 100
1100 150
1100 200
1100 250
1100 300
1100 350
1100 400
1100 450
1100 500
1100 550
1100 600
1100 650
1100 700
1100 750
1100 800
1100 850
1200 50
1200 100
1200 150
1200 200
1200 250
1200 300
1200 350
1200 400
1200 450
1200 500
1200 550
1200 600
1200 650
1200 700
1200 750
1200 800
1200 850
1200 900
1200 950


KeyError: -1

In [68]:
# A
Battery = init_SOC*E_B
SOC=[init_SOC]
cost = []
cost_PV_50_100 = []
cost_WG_50_100 = []
cost_net_50_100 = []
sum_A = Cost_Battery_daily
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_A[t], P_PV_A[t], 0,"A",t)
    # print(C_light, C_wind, C_net)
    cost_PV_50_100.append(C_light*0.4)
    cost_WG_50_100.append(C_wind*0.5)
    cost_net_50_100.append(C_net*1)
    cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
    sum_A+=cost[t] 
    # print(cost[t])

df = pd.DataFrame({"cost_PV":cost_PV_50_100,"cost_WG":cost_WG_50_100,"cost_net":cost_net_50_100,"cost_sum":cost})
df.to_csv("data/cost_A_50_100",index=False)

# sum_A = sum(cost_A)
# print(cost_A)
# print(sum_A)

外部放电38.0kW,消耗电池40.0
外部充电50.0kW,补充电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电50.0kW,消耗电池52.63157894736842
外部放电26.000000000000007kW,消耗电池27.368421052631586
外部充电3.95516952522712e-15kW,补充电池4.163336342344337e-15


In [69]:
# B
Battery = init_SOC*E_B
SOC=[init_SOC]
cost = []
cost_PV_50_100 = []
cost_WG_50_100 = []
cost_net_50_100 = []
sum_B = Cost_Battery_daily
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_B[t], 0,P_WG_B[t],"B",t)
    # print(C_light, C_wind, C_net)
    cost_PV_50_100.append(C_light*0.4)
    cost_WG_50_100.append(C_wind*0.5)
    cost_net_50_100.append(C_net*1)
    cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
    # print(cost[t])
    sum_B+=cost[t]

df = pd.DataFrame({"cost_PV":cost_PV_50_100,"cost_WG":cost_WG_50_100,"cost_net":cost_net_50_100,"cost_sum":cost})
df.to_csv("data/cost_B_50_100",index=False)
# sum_B = sum(cost_B)
# print(cost_B)
# print(sum_B)

外部放电38.0kW,消耗电池40.0
外部充电27.099999999999998kW,补充电池25.744999999999997
外部放电24.457749999999997kW,消耗电池25.744999999999997
外部充电49.99999999999999kW,补充电池47.49999999999999
外部放电45.125kW,消耗电池47.5
外部充电12.925000000000006kW,补充电池12.278750000000006
外部放电11.6648125kW,消耗电池12.278749999999999
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电30.875000000000007kW,消耗电池32.50000000000001
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电30.875000000000007kW,消耗电池32.50000000000001
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5


In [70]:
# C
Battery = init_SOC*E_B
cost= []
cost_PV_50_100 = []
cost_WG_50_100 = []
cost_net_50_100 = []
sum_C = Cost_Battery_daily
SOC=[init_SOC]
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_C[t], P_PV_C[t],P_WG_C[t],"C",t,debug=False) 
    # print(C_light, C_wind, C_net)
    cost_PV_50_100.append(C_light*0.4)
    cost_WG_50_100.append(C_wind*0.5)
    cost_net_50_100.append(C_net*1)
    cost.append(C_light*0.4+C_wind*0.5+C_net*1) 
    # print(cost[t])
    sum_C+=cost[t]
# print(cost_C)
# print(sum(cost_C))
# print(sum_C)
df = pd.DataFrame({"cost_PV":cost_PV_50_100,"cost_WG":cost_WG_50_100,"cost_net":cost_net_50_100,"cost_sum":cost})
df.to_csv("data/cost_C_50_100",index=False)


# bug:
# 电池只有0.1时还在放电，应该终止

外部放电38.0kW,消耗电池40.0
外部充电50.0kW,补充电池47.5
外部放电45.125kW,消耗电池47.5
外部充电50.0kW,补充电池47.5
外部充电34.21052631578948kW,补充电池32.50000000000001
外部放电10.739999999999899kW,消耗电池11.305263157894629
外部放电6.0199999999999845kW,消耗电池6.336842105263141
外部放电50.0kW,消耗电池52.63157894736842
外部充电11.900277008310136kW,补充电池11.305263157894629
外部放电26.000000000000007kW,消耗电池27.368421052631586
外部充电3.95516952522712e-15kW,补充电池4.163336342344337e-15


In [73]:
# 创建 DataFrame
df = pd.DataFrame({'cost_A': cost_A, 'cost_B': cost_B, 'cost_C': cost_C})

# 将 DataFrame 写入 CSV 文件
df.to_csv('data/cost_50_100.csv',index=False)

In [76]:
cost_0_0 = pd.read_csv("data/cost_0_0.csv")
cost_A_0_0 = cost_0_0["cost_A"]
cost_B_0_0 = cost_0_0["cost_B"]
cost_C_0_0 = cost_0_0["cost_C"]
print(sum(cost_A_0_0),sum(cost_B_0_0),sum(cost_C_0_0))
cost_A_50_100 = pd.read_csv("data/cost_A_50_100")
cost_B_50_100 = pd.read_csv("data/cost_B_50_100")
cost_C_50_100 = pd.read_csv("data/cost_C_50_100")
print(sum(cost_A_50_100["cost_sum"])+Cost_Battery_daily,sum(cost_B_50_100["cost_sum"])+Cost_Battery_daily,sum(cost_C_50_100["cost_sum"])+Cost_Battery_daily)
print(
    sum(cost_A_50_100["cost_sum"])-sum(cost_A_0_0),
    sum(cost_B_50_100["cost_sum"])-sum(cost_B_0_0),
    sum(cost_C_50_100["cost_sum"])-sum(cost_C_0_0))



6084.875 5071.15 4963.311000000001
6031.148972602739 5063.223217668529 4786.550111106895
-114.0 -68.20075493420973 -237.03486149584478


In [74]:
cost_0_0 = pd.read_csv("data/cost_0_0.csv")
cost_A_0_0 = cost_0_0["cost_A"]
cost_B_0_0 = cost_0_0["cost_B"]
cost_C_0_0 = cost_0_0["cost_C"]
cost_50_100 = pd.read_csv("data/cost_50_100.csv")
cost_A_50_100 = cost_50_100["cost_A"]
cost_B_50_100 = cost_50_100["cost_B"]
cost_C_50_100 = cost_50_100["cost_C"]
Delta_A_50_100 = cost_A_50_100 - cost_A_0_0
Delta_B_50_100 = cost_B_50_100 - cost_B_0_0
Delta_C_50_100 = cost_C_50_100 - cost_C_0_0
df = pd.DataFrame({"Delta_A_50_100":Delta_A_50_100,"Delta_B_50_100":Delta_B_50_100,"Delta_C_50_100":Delta_C_50_100})
df.to_csv('data/Delta_50_100.csv',index=False)
sum_A_0_0 = sum(cost_A_0_0)
sum_B_0_0 = sum(cost_B_0_0)
sum_C_0_0 = sum(cost_C_0_0)
sum_A_50_100 = sum(cost_A_50_100)
sum_B_50_100 = sum(cost_B_50_100)
sum_C_50_100 = sum(cost_C_50_100)

print(sum_A_0_0,sum_B_0_0,sum_C_0_0)
print(sum_A_50_100,sum_B_50_100,sum_C_50_100)
print(sum(Delta_A_50_100),sum(Delta_B_50_100),sum(Delta_C_50_100))

6084.875 5071.15 4963.311000000001
5970.875 5002.94924506579 6167.075000000001
-114.0 -68.2007549342105 1203.764


400 50
外部放电49.99999999999999kW,消耗电池52.63157894736841
外部放电50.0kW,消耗电池52.63157894736842
外部放电50.0kW,消耗电池52.63157894736842
外部放电2.000000000000022kW,消耗电池2.1052631578947603


KeyError: -1